In [1]:
pip install ftfy regex tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 947.2 kB/s eta 0:00:00


In [2]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xaag71wx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-xaag71wx
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
import torch
import clip
from PIL import Image

In [4]:
# Load the CLIP model and preprocessor
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 133MiB/s]


In [5]:
# List of sentences for each organ
sentences = {
    'liver':["liver",
    "The liver appears as a large, wedge-shaped organ with a smooth convex superior surface and a concave inferior surface in scans",
    "It exhibits a homogeneous texture composed mainly of hepatocytes, making it distinguishable in imaging studies",
    "The liver is easily visible in both CT and MRI scans due to its size and location in the upper right abdomen",
    "It occupies a significant portion of the upper right quadrant of the abdomen",
    "The liver is reddish-brown in color and has a smooth surface",
    "It stands out due to its unique shape and relatively high contrast with other abdominal structures",
    "In contrast-enhanced scans, the liver's vascular structures become more apparent, aiding in identification",
    "The liver's smooth texture sets it apart from neighboring organs like the spleen and kidneys",
    "Its location in the upper right abdomen and proximity to the ribcage help distinguish it from the background",
    "Fatty liver disease can alter the liver's appearance in scans due to the accumulation of fat within the organ"
    ],  # Your liver sentences here

    'left_kidney':  ["left kidney",
    "The left kidney appears as a bean-shaped organ in abdominal scans",
    "It has a smooth convex outer surface and a concave inner surface called the renal hilum",
    "The left kidney is clearly visible in CT and ultrasound scans",
    "It is situated below the diaphragm and behind the stomach on the left side of the abdomen",
    "The left kidney has a reddish-brown color and a smooth texture",
    "Its distinct bean-like shape sets it apart from other abdominal structures",
    "In CT scans, the renal hilum and internal structures help identify the left kidney",
    "Its unique location and proximity to the spleen and stomach aid in distinguishing it from the background",
    "Ultrasound imaging enables real-time visualization of the left kidney's movement, helping to distinguish it from surrounding tissues",
    "Left kidney pain can be a symptom of kidney infection or stones, prompting the need for medical attention"
    ],    # Your left kidney sentences here

    'right_kidney':  ["right kidney",
    "The right kidney appears as a bean-shaped organ in scans",
    "It has a smooth convex outer surface and a concave inner surface called the renal hilum",
    "The right kidney is clearly visible in CT and ultrasound scans",
    "It is situated slightly lower than the left kidney on the right side of the abdomen",
    "The right kidney has a reddish-brown color and a smooth texture",
    "Its unique shape and location help differentiate it from surrounding structures",
    "In CT scans, the renal hilum and internal structures aid in distinguishing the right kidney",
    "Its proximity to the liver, gallbladder, and colon helps locate the right kidney in scans",
    "In ultrasound, the right kidney's movement during breathing helps distinguish it from adjacent tissues,",
    "Right kidney pain can be a symptom of kidney infection or stones, necessitating medical evaluation"
    ],  # Your right kidney sentences here

    'spleen': ["spleen",
    "The spleen appears as a flat, oval-shaped organ in scans, located in the upper left abdomen",
    "It typically has an oval or flattened shape, varying in size among individuals",
    "The spleen is visible in both CT and ultrasound scans, aiding in its evaluation",
    "It occupies a significant portion of the upper left quadrant of the abdomen",
    "The spleen has a reddish-brown color and a smooth surface",
    "Its unique shape and location in the upper left abdomen help distinguish it from other organs",
    "In CT scans, its internal structures and blood vessels help identify the spleen",
    "In ultrasound, the spleen's texture and movement during breathing differentiate it from surrounding tissues",
    "Enlargement of the spleen (splenomegaly) is evident in scans and can indicate certain medical conditions",
    "Medical imaging can be used to assess the spleen's health, detect any abnormalities or enlargement, and guide appropriate treatment"
    ],  # Your spleen sentences here
}

# Create lists to store the mean text embeddings for each organ
organ_vectors =[]

# Process sentences and calculate mean text embeddings for each organ
for organ, organ_sentences in sentences.items():
    text_input = torch.cat([clip.tokenize(sent) for sent in organ_sentences]).to(device)

    # Get the text embeddings
    with torch.no_grad():
        text_features = model.encode_text(text_input)

    # Take the mean of the text embeddings (num_sentences, 512) along the first dimension (rows)
    mean_text_features = torch.mean(text_features, dim=0)

#     # Append the mean_text_features to the corresponding list
    organ_vectors.append(mean_text_features)

# Save the vectors to a .pt file
output_path = "organ_vectors_only.pt"
torch.save(organ_vectors, output_path)